# Notebook to Convert AmsterdamUMCdb to MIMIC-III Format

Here we will be converting the [AmsterdamUMCdb](https://github.com/AmsterdamUMC/AmsterdamUMCdb) data files to the MIMIC-III data file format as generated by [MIMIC-Code](https://github.com/MIT-LCP/mimic-code). We do this as to allow the exact same preprocessing pipeline to be applied to both MIMIC and the AmsterdamUMCdb.

In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

CHUNK_SIZE = 20000
MAX_CHUNKS = 100000000  # for debugging
OUT_DIR = 'final/'

### Convenience functions

As the files are in the orders of GBs we read in and process the files incrementally;

In [ ]:
# reads file from path in chunks of size `chunksize`
def read_csv(path, usecols, chunksize=CHUNK_SIZE):
    for i, chunk in enumerate(pd.read_csv(path, usecols=usecols, encoding='latin1', engine='python', chunksize=chunksize)):
        yield i, chunk.reset_index(drop=True) # resets index so that indices range from 0 to chunksize - 1

Also, we use a progressbar which shows the number of admissions processed so far and how many there are still to go with an estimated processing time;

In [ ]:
def pbar(iterator, total_admissions=23106):
    # Keep track of admissions already seen
    processed_admissions = set()
    
    with tqdm(total=total_admissions) as progress_bar:
        for i, chunk in iterator:
            
            # count number of new admissions not yet seen
            # we do it only if last admission is new (saves time checking)
            if chunk.admissionid.values[-1] not in processed_admissions:
                new_admissions = set(chunk.admissionid) - processed_admissions
                processed_admissions.update(new_admissions)
            
                # update progress bar
                progress_bar.update(len(new_admissions))
                
            yield i, chunk

---
### Times
Start of admission is in milliseconds relative to the first admission of that same patient; therefore we convert to hours 

In [ ]:
# Start times of admissions relative to the first admission (with start = 0)
start_of_admission = dict()
for _, chunk in read_csv(r"D:/AmsterdamUMCdb-v1.0.2/admissions.csv", usecols=['admissionid', 'admittedat']):
    # admissionid + admission time
    start_of_admission.update(dict(zip(chunk.admissionid, chunk.admittedat)))

print('Time since last admission: %.2f years' % (start_of_admission[14] / (1000 * 60 * 60 * 24 * 365)))

In [ ]:
# convenience function to compute time from start of admission in hours
# Note: original times are relative to first admission of same patient!
def time_within_admission(times, admissionids):
    starttimes = pd.Series([start_of_admission[a] for a in admissionids])
    return (times.reset_index(drop=True) - starttimes) / (60*60*1000)  # hours


# create a timestamp from relative hours relative to some starttime or anchor
def hours_to_absolute_time(anchor, hours):
    return anchor + pd.to_timedelta(arg=(hours * 3600 * 1000).astype(int), unit='ms')

----

## Demographics

#### Systemic inflammatory response syndrome (SIRS) score
Sources:
- Temperature: [temperature.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/temperature.sql)
- Heart rate: [heart_rate.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/heart_rate.sql)
- Respiratory rate: [resp_rate.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/resp_rate.sql)
- WBC/Leukocytes: [wbc.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/wbc.sql)
- SIRS definition: https://www.mdcalc.com/calc/1096/sirs-sepsis-septic-shock-criteria

In [ ]:
TEMPERATURE = [
    8658,  # --Temp Bloed
    8659,  # --Temperatuur Perifeer 2
    8662,  # --Temperatuur Perifeer 1
    13058, # --Temp Rectaal
    13059, # --Temp Lies
    13060, # --Temp Axillair
    13061, # --Temp Oraal
    13062, # --Temp Oor
    13063, # --Temp Huid
    13952, # --Temp Blaas
    16110  # --Temp Oesophagus
]

PCO2 = [
    6846,  # --PCO2
    9990,  # --pCO2 (bloed)
    21213  # --PCO2 (bloed) - kPa
]

HEARTRATE = [
    6640   # --Hartfrequentie
]

RESP_RATE = [
    8873,  # --Ademfrequentie Evita: measurement by Evita ventilator, most accurate
    12266, # --Ademfreq.: measurement by Servo-i/Servo-U ventilator, most accurate
    8874   # --Ademfrequentie Monitor: measurement by patient monitor using ECG-impedance, less accurate  
]

WBC = [
    6779,  # --Leucocyten 10^9/l
    9965   # --Leuco's (bloed) 10^9/l
]

BANDS = [
    11586  # -- Staaf % (bloed)
]

For each chunk of the `numericitems.csv` with a few admissions, we will extract the above parameters (incl. temperature, heart rate, PCO2, leukocyte count, etc.) and infer their earliest values within the admission. We limit these values to at most 8 hours after the start of admission (see Roggeveen et al. (2021)). We then compute the SIRS score from those parameters.

In [ ]:
def compute_sirs(resp_rate, paco2, temp, heart_rate, wbc, bands):
    """ Computes Systemic inflammatory response syndrome (SIRS) score given vital parameters """
    # which admissions have available the data to estimate SIRS?
    admissionids = temp.keys() & heart_rate.keys() & (resp_rate.keys() | paco2.keys()) & (wbc.keys() | bands.keys())
    
    # compute SIRS for eligible admissions
    out = dict()
    for admissionid in admissionids:
        # For details, see: http://www.mdcalc.com/calc/1096/sirs-sepsis-septic-shock-criteria
        score = int(temp[admissionid] < 36 or temp[admissionid] > 38)
        score += int(heart_rate[admissionid] > 90)
        score += int(resp_rate[admissionid] > 20) if admissionid in resp_rate else int(paco2[admissionid] < 32)
        score += int(wbc[admissionid] > 12 or wbc[admissionid] < 4) if admissionid in wbc else int(bands[admissionid] > 10)
        
        out[admissionid] = score
    return out

In [ ]:
sirs_on_admission = dict()

for i, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/numericitems.csv", usecols=['admissionid', 'itemid', 'value', 'measuredat', 'unitid'])):
    
    # which observations are within 24h of start admission?
    times = time_within_admission(chunk.measuredat, chunk.admissionid)
    within_24h = (times > -0.5) & (times < 8) # Allow 30 minute margin before admission
    chunk = chunk[within_24h]
    
    if len(chunk) == 0:
        continue
    
    # Respiratory rate (unit = breaths/min)
    resp_rate = chunk[chunk.itemid.isin(RESP_RATE) & (chunk.unitid == 15)]
    resp_rate = resp_rate.groupby('admissionid', sort=False).value.first().to_dict()
    
    # PaCO2 (mmHg)
    paco2 = chunk[chunk.itemid.isin(PCO2)].copy()
    paco2.loc[paco2.unitid == 152, 'value'] *= paco2.value * 7.50061683  # conversion rate kPa to mmHg
    paco2 = paco2.groupby('admissionid', sort=False).value.first().to_dict()
    
    # Temperature (degrees C)
    temp = chunk[chunk.itemid.isin(TEMPERATURE) & (chunk.unitid == 59)]
    temp = temp.groupby('admissionid', sort=False).value.first().to_dict()
    
    # Heart rate (beats/min)
    heart_rate = chunk[chunk.itemid.isin(HEARTRATE) & (chunk.unitid == 15)]
    heart_rate = heart_rate.groupby('admissionid', sort=False).value.first().to_dict()
    
    # Leukocytes (10^9/l)
    wbc = chunk[chunk.itemid.isin(WBC) & (chunk.unitid == 101)]
    wbc = wbc.groupby('admissionid', sort=False).value.first().to_dict()
    
    # Bands (bloed %)
    bands = chunk[chunk.itemid.isin(BANDS)]
    bands = bands.groupby('admissionid', sort=False).value.first().to_dict()
        
    # SIRS score
    sirs_on_admission.update(compute_sirs(resp_rate, paco2, temp, heart_rate, wbc, bands))
    
    if i == MAX_CHUNKS:
        break

In [ ]:
sirs_on_admission[11]  # SIRS of admission 11 in first 24h

#### Mechanical ventilation

Source: [mechanical_ventilation.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/lifesupport/mechanical_ventilation.sql)

In [ ]:
# Combinations of itemids and valueids in listitems.csv
# indicating use of (non-)invasive mechanical ventilation
MECH_VENT_SETTINGS = [
    (9534, (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13)),                    # --Type beademing Evita 1
    (6685, (1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 20, 22)),                 # --Type Beademing Evita 4
    (8189, (16,)),                                                          # --Toedieningsweg O2
    (12290, (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18)),  # --Ventilatie Mode (Set) Servo-I and Servo-U ventilators
    (12347, (2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18)),  # --Ventilatie Mode (Set) (2) Servo-I and Servo-U ventilators
    (12376, (1, 2)),                                                        # --Mode (Bipap Vision)
]    

In [ ]:
class Ventilation():
    """
    Convenience class to check for mechanical ventilation
    """
    def __init__(self, mech_vent):
        self._vent_settings = set()
        for itemid, valueids in mech_vent:
            self._vent_settings.update([(itemid, valueid) for valueid in valueids])
            
    def __call__(self, itemids, valueids):
        """ Checks if (itemid, valueid) pair is a valid ventilator settings """
        return np.array([x in self._vent_settings for x in list(zip(itemids, valueids))])
    
is_mech_vent = Ventilation(MECH_VENT_SETTINGS)

For each admission, see if mechanical ventilation is used in the first 24 hours (see Roggeveen et al., 2021)

In [ ]:
admissions_with_vent = set()

for _, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/listitems.csv", usecols=['admissionid', 'itemid', 'valueid', 'measuredat'])):
    # which observations are within 24h of start admission?
    times = time_within_admission(chunk.measuredat, chunk.admissionid)
    within_24h = (times > -0.5) & (times < 8) # Allow 30 minute margin before admission
    
    # check whether (itemid, valueid) in listitems is mechanical ventilation and whether it's in the first 24 hours
    chunk['has_vent'] = is_mech_vent(chunk.itemid, chunk.valueid) & within_24h
    
    # admissionids with mechanical ventilation
    admissions_with_vent.update(chunk.admissionid[chunk.has_vent])
        
print('Number of admissions with ventilation:', len(admissions_with_vent))

#### Glasgow Coma Scale
Sources: 
- Glasgow Coma Scale: [gcs.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/gcs.sql)
- GCS definition: https://www.mdcalc.com/calc/64/glasgow-coma-scale-score-gcs

In [ ]:
## credits Patrick Thoral

def gcs_eyes_score(df):
    df = df.copy()
    df.loc[df.itemid == 6732, 'eyes_score'] = 5 - df.valueid   # --Actief openen van de ogen
    df.loc[df.itemid == 13077, 'eyes_score'] = df.valueid      # --A_Eye
    df.loc[df.itemid == 14470, 'eyes_score'] = df.valueid - 4  # --RA_Eye
    df.loc[df.itemid == 16628, 'eyes_score'] = df.valueid - 4  # --MCA_Eye
    df.loc[df.itemid == 19635, 'eyes_score'] = df.valueid - 4  # --E_EMV_NICE_24uur
    df.loc[df.itemid == 19638, 'eyes_score'] = df.valueid - 8  # --E_EMV_NICE_Opname
    return df
    
def gcs_motor_score(df):
    df = df.copy()
    df.loc[df.itemid == 6734, 'motor_score'] = 7 - df.valueid   # --Beste motore reactie van de armen
    df.loc[df.itemid == 13072, 'motor_score'] = df.valueid      # --A_Motoriek
    df.loc[df.itemid == 14476, 'motor_score'] = df.valueid - 6  # --RA_Motoriek
    df.loc[df.itemid == 16634, 'motor_score'] = df.valueid - 6  # --MCA_Motoriek
    df.loc[df.itemid == 19636, 'motor_score'] = df.valueid - 6  # --M_EMV_NICE_24uur
    df.loc[df.itemid == 19639, 'motor_score'] = df.valueid - 12 # --M_EMV_NICE_Opname
    return df

def gcs_verbal_score(df):
    df = df.copy()
    df.loc[df.itemid == 6735, 'verbal_score'] = 6 - df.valueid   # --Beste verbale reactie
    df.loc[df.itemid == 13066, 'verbal_score'] = df.valueid      # --A_Verbal
    df.loc[df.itemid == 14482, 'verbal_score'] = df.valueid - 5  # --RA_Verbal
    df.loc[df.itemid == 16640, 'verbal_score'] = df.valueid - 5  # --MCA_Verbal
    df.loc[df.itemid == 19637, 'verbal_score'] = df.valueid - 9  # --V_EMV_NICE_24uur
    df.loc[df.itemid == 19640, 'verbal_score'] = df.valueid - 15 # --M_EMV_NICE_Opname
    
    # cap verbal score at 1
    df.loc[df['verbal_score'] < 1, 'verbal_score'] = 1
    return df

In [ ]:
gcs_on_admission = dict()

for i, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/listitems.csv", usecols=['admissionid', 'itemid', 'valueid', 'value', 'measuredat'])): 
    # which observations are within 24h of start admission?
    times = time_within_admission(chunk.measuredat, chunk.admissionid)
    within_24h = (times > -0.5) & (times < 8) # Allow 30 minute margin before admission
    
    # component scores
    chunk = gcs_eyes_score(chunk)
    chunk = gcs_motor_score(chunk)
    chunk = gcs_verbal_score(chunk)
        
    # GCS = eye + motor + verbal score
    gcs = chunk.groupby('admissionid').first()[['eyes_score', 'motor_score', 'verbal_score']].sum(axis=1, min_count=1)
        
    # drop admissions where score could not be computed
    gcs = gcs[gcs.notna()].to_dict()
    
    gcs_on_admission.update(gcs)
    
    if i % 1000 == 0:
        print('(%d/23106) number of GCS computed: %s' % (i, len(gcs_on_admission)))

In [ ]:
gcs_on_admission[11]

#### Sequential Organ Failure Assessment (SOFA) score

Sources:

- PaO2/FiO2: [pO2_pCO2_FiO2.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/pO2_pCO2_FiO2.sql)
- Ventilators: [mechanical_ventilation.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/lifesupport/mechanical_ventilation.sql)
- Platelets: [platelets.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/platelets.sql)
- Glasgow Coma Scale: [gcs.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/gcs.sql)
- Bilirubin: [bilirubin.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/b6f295fbfbd9a1f8f9cfa71dab58c670120543d8/amsterdamumcdb/sql/common/bilirubin.sql)
- SOFA definition: https://www.mdcalc.com/calc/691/sequential-organ-failure-assessment-sofa-score

In [ ]:
PO2 = [
    7433,  # --PO2
    9996,  # --PO2 (bloed)
    21214  # --PO2 (bloed) - kPa
]

# TODO: estimate FiO2 if not known from ventilator settings
FIO2 = [
    6699,  # --FiO2 %: setting on Evita ventilator
    12279, # --O2 concentratie --measurement by Servo-i/Servo-U ventilator
    12369, # --SET %O2: used with BiPap Vision ventilator
    16246  # --Zephyros FiO2: Non-invasive ventilation
]

PLATELETS = [
    9964,  # --Thrombo's (bloed)
    6797,  # --Thrombocyten
    10409, # --Thrombo's citr. bloed (bloed)
    14252  # --Thrombo CD61 (bloed)   
]

BILIRUBIN = [
    6813,  # --Bili Totaal
    9945   # --Bilirubine (bloed)
]

MEANBP = [
    6642,  # --ABP gemiddeld
    6679,  # --Niet invasieve bloeddruk gemiddeld
    8843   # --ABP gemiddeld II
]

CREATININE = [
    6836,  # --Kreatinine µmol/l (erroneously documented as µmol)
    9941,  # --Kreatinine (bloed) µmol/l
    14216  # --KREAT enzym. (bloed) µmol/l   
]

In [ ]:
def compute_sofa(po2, fio2, vent, platelets, gcs, bilirubin, meanbp, creatinine):
    """ Computes Sequential Organ Failure Assessment (SOFA) score given vital parameters 
    For details, see: https://files.asprtracie.hhs.gov/documents/aspr-tracie-sofa-score-fact-sheet.pdf
    """
    # which admissions have available the data to estimate SOFA?
    admissionids = po2.keys() & fio2.keys() & vent.keys() & platelets.keys() & bilirubin.keys() & meanbp.keys() & creatinine.keys() & gcs.keys()
    
    # compute SIRS for eligible admissions
    out = dict()
    for admissionid in admissionids:
        score = 0
        
        # respiratory
        pf_ratio = po2[admissionid] / fio2[admissionid]
        if pf_ratio < 100 and vent[admissionid]:
            score += 4
        elif pf_ratio < 200 and vent[admissionid]:
            score += 3
        elif pf_ratio < 300:
            score += 2
        elif pf_ratio < 400:
            score += 1
            
        # coagulation
        coag = platelets[admissionid]
        if coag < 20:
            score += 4
        elif coag < 50:
            score += 3
        elif coag < 100:
            score += 2
        elif coag < 150:
            score += 1
            
        # liver
        bili = bilirubin[admissionid] 
        if bili >= 12:
            score += 4
        elif 6 <= bili < 12:
            score += 3
        elif 2 <= bili < 6:
            score += 2
        elif 1.2 <= bili < 2:
            score += 1
            
        # cardiovascular
        # TODO: add vasopressor agents
        blood_pressure = meanbp[admissionid] 
        if blood_pressure < 70:
            score += 1
        
        # central nervous system
        gcs_score = gcs[admissionid]
        if gcs_score < 6:
            score += 4
        elif 6 <= gcs_score <= 9:
            score += 3
        elif 10 <= gcs_score <= 12:
            score += 2
        elif 13 <= gcs_score <= 14:
            score += 1
            
        # renal
        creat = creatinine[admissionid]
        if creat > 440:
            score += 4
        elif 300 <= creat < 440:
            score += 3
        elif 170 <= creat < 300:
            score += 2
        elif 110 <= creat < 170:
            score += 1
        
        out[admissionid] = score
    return out
    

In [ ]:
sofa_on_admission = dict()

for i, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/numericitems.csv", usecols=['admissionid', 'itemid', 'value', 'measuredat', 'unitid', 'unit'])):
    # which observations are within 8h after start admission?
    times = time_within_admission(chunk.measuredat, chunk.admissionid)
    chunk = chunk[(times > -0.5) & (times < 8)].copy()  # Allow 30 minute margin before admission
    
    if len(chunk) == 0:
        continue
    
    # PO2 (mmHg)
    po2 = chunk[chunk.itemid.isin(PO2)].copy()
    po2.loc[po2.unitid == 152, 'value'] *= po2.value * 7.50061683  # conversion rate kPa to mmHg
    po2 = po2.groupby('admissionid', sort=False).value.first().to_dict()
    if len(po2) == 0:
        continue
    
    # FiO2 (%)
    fio2 = chunk[chunk.itemid.isin(FIO2)]
    fio2 = fio2.groupby('admissionid', sort=False).value.first().to_dict()
    if len(fio2) == 0:
        continue
    
    # Mechanical ventilation
    vent = chunk.admissionid.isin(admissions_with_vent).to_dict()
    
    # Platelets (10^9/l)
    platelets = chunk[chunk.itemid.isin(PLATELETS)]
    platelets = platelets.groupby('admissionid', sort=False).value.first().to_dict()
    if len(platelets) == 0:
        continue
    
    # Bilirubin (µmol/l)
    bilirubin = chunk[chunk.itemid.isin(BILIRUBIN)]
    bilirubin = bilirubin.groupby('admissionid', sort=False).value.first().to_dict()
    if len(bilirubin) == 0:
        continue
    
    # MAP (mmHg)
    meanbp = chunk[chunk.itemid.isin(MEANBP)]
    meanbp = meanbp.groupby('admissionid', sort=False).value.first().to_dict()
    if len(meanbp) == 0:
        continue
    
    # Creatinine (µmol/l)
    creatinine = chunk[chunk.itemid.isin(CREATININE)]
    creatinine = creatinine.groupby('admissionid', sort=False).value.first().to_dict()
    
    sofa_on_admission.update(compute_sofa(po2, fio2, vent, platelets, gcs_on_admission, bilirubin, meanbp, creatinine))
    
    if i % 500 == 0:
        print('(%d/23106) number of SOFA computed: %s' % (i, len(sofa_on_admission)))

#### Age, Height, Weight, Gender

In [ ]:
# Age/weight/height groups to their centroids (best guess)
AGE_GROUPS = {
    '18-39': 30,
    '40-49': 45,
    '50-59': 55,
    '60-69': 65,
    '70-79': 75, 
    '80+': 85  # Arbitrary
}

WEIGHT_GROUPS = {
    '59-': 55,
    '60-69': 65, 
    '70-79': 75,
    '80-89': 85,
    '90-99': 95,
    '100-109': 105,
    '110+': 115,
    'N/A': -1  # To be replaced with Dutch average weight (for men and women separately)
}
  
HEIGHT_GROUPS = {
    '159-': 155,
    '160-169': 165,
    '170-179': 175,
    '180-189': 185, 
    '190+': 195, 
    'N/A': -1  # Average height of men and women in NL
}

Using the above *mapper* functions we can now map from grouped demographics (`agegroup`, `heightgroup`, `weightgroup`) to a real value and when unknown use per-gender guesses of weight and height (better than a naive average over the population)

### Demographics DataFrame

In [ ]:
demo_cols = ['admissionid', 'gender', 'agegroup', 'heightgroup', 'weightgroup']

demo_df = []
for i, chunk in read_csv(r"D:/AmsterdamUMCdb-v1.0.2/admissions.csv", usecols=demo_cols, chunksize=100000000):
    demo_chunk = pd.DataFrame({
        'icustay_id': chunk.admissionid,
        'is_male': chunk.gender == 'Man',
        'age': chunk.agegroup.transform(lambda x: AGE_GROUPS[x] if x in AGE_GROUPS else np.NaN),
        'height': chunk.heightgroup.transform(lambda x: HEIGHT_GROUPS[x] if x in HEIGHT_GROUPS else np.NaN),
        'weight': chunk.weightgroup.transform(lambda x: WEIGHT_GROUPS[x] if x in WEIGHT_GROUPS else np.NaN),
        'vent': chunk.admissionid.isin(admissions_with_vent),
        'sirs': chunk.admissionid.transform(lambda x: sirs_on_admission[x] if x in sirs_on_admission else np.NaN),  # NaN if couldn't be estimated
        'sofa': chunk.admissionid.transform(lambda x: sofa_on_admission[x] if x in sofa_on_admission else np.NaN)
    })
    demo_df.append(demo_chunk)
    
demo_df = pd.concat(demo_df, axis=0).reset_index(drop=True)

In [ ]:
demo_df

---

## Vitals

Sources:
- Vital parameters: [amsterdamumcdb/sql/common](https://github.com/AmsterdamUMC/AmsterdamUMCdb/tree/master/amsterdamumcdb/sql/common)

In [ ]:
VITALS = {
    'HeartRate': [6640],                                     # 'Hartfrequentie'
    'SysBP':     [6641, 6678, 8841],                         # 'ABP systolisch', 'Niet invasieve bloeddruk systolisch', 'ABP systolisch II', 
    'DiasBP':    [6643, 6680, 8842],                         # 'ABP diastolisch', 'Niet invasieve bloeddruk diastolisch', 'ABP diastolisch II'
    'MeanBP':    [6642, 6679, 8843],                         # 'ABP gemiddeld', 'Niet invasieve bloeddruk gemiddeld', 'ABP gemiddeld II'
    'Glucose':   [6833],                                     # 'Glucose Bloed'
    'SpO2':      [12311],                                    # 'O2-Saturatie (bloed)'
    'TempC':     [8658, 8659, 8662, 13058, 13059,            # 'Temp Bloed', 'Temperatuur Perifeer 2', 'Temperatuur Perifeer 1', 'Temp Rectaal', 'Temp Lies',
                 13060, 13061, 13062, 13063, 13952, 16110],  # 'Temp Axillair', 'Temp Oraal', 'Temp Oor', 'Temp Huid', 'Temp Blaas', 'Temp Oesophagus' 
    'RespRate':  [8874, 8873, 9654, 7726, 12266]             # 'Ademfrequentie Monitor', 'Ademfrequentie Evita', 'Ademfreq.'
}

In [ ]:
vitals_df = []

for i, chunk in tqdm(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/numericitems.csv", usecols=['admissionid', 'itemid', 'value', 'measuredat'])):
    
    for vital, vital_ids in VITALS.items():
        # Mask entries of vital
        is_vital = chunk.itemid.isin(vital_ids)
        
        if is_vital.any():
            
            # Store measurements of vital in new DataFrame
            vital_df = pd.DataFrame({
                'icustay_id': chunk.admissionid[is_vital].values,
                'charttime': time_within_admission(chunk.measuredat[is_vital], chunk.admissionid[is_vital]),
                'vital_id': vital,
                'valuenum': chunk.value[is_vital].values,
            })
            
            vitals_df.append(vital_df)
        
    if i > MAX_CHUNKS:
        break
        
# Merge vital DataFrames
vitals_df = pd.concat(vitals_df, axis=0).reset_index(drop=True)
vitals_df.head()

In [ ]:
# save!
vitals_df.to_csv(OUT_DIR + 'vitals_cohort.csv')

---

## Lab Results
Sources: lots of hard work, blood, sweat and tears

In [ ]:
LABRESULTS = {
    'Calcium':     [9933],                           # 'Calcium', 'Calcium totaal (bloed)'
    'IonCalcium':  [],
    'ASAT':        [11990],                          # 'ASAT (bloed)'
    'PTT':         [17982],                          # 'APTT (bloed)'
    'Potassium':   [9927],                           # 'Kalium (bloed)'
    'PT':          [],
    'Platelet':    [6797, 9964, 10409, 14252, 7369], # 'Thrombocyten', "Thrombo's (bloed)", "Thrombo's citr. bloed (bloed)", 'Thrombo CD61 (bloed)'
    'AnionGap':    [9559],                           # 'Anion-Gap (bloed)'
    'PaO2':        [7433, 9996, 21214],              # 'PaO2', 'PaO2 (bloed)', 'PaO2 (bloed) - kPa'
    'ALAT':        [6800, 11978],                    # 'ALAT', 'ALAT (bloed)'
    'WBC':         [11678, 18553, 18554,             # 'Tot.WBC*10^8 (overig)', 'Tot. WBC*10^8 (HPC-A Allogeen)', 'Tot. WBC*10^8 (HPC-A Autoloog)'
                    18557, 6779, 9965],              # 'Tot. WBC*10^8 (Tcellen concentraat)', 'Leucocyten 10^9/l', "Leuco's (bloed) 10^9/l"
    'Bilirubin':   [9945, 6813],                     # 'Bilirubine (bloed)', 'Bili Totaal'
    'Sodium':      [12233, 9555, 9924, 10284],       # 'Natrium (overig)', 'Natrium Astrup', 'Natrium (bloed)', 'Na (onv.ISE) (bloed)'
    'Chloride':    [],
    'Magnesium':   [9952],                           # 'Magnesium (bloed)'
    'Lactate':     [10053],                          # 'Lactaat (bloed)'
    'PaCO2':       [6846, 9990, 21213],              # 'PCO2', 'PCO2 (bloed)', 'PCO2 (bloed) - kPa'
    'Glucose':     [6833, 9947],                     # 'Glucose Bloed', 'Glucose (bloed)'
    'Creatinine':  [9941, 14216],                    # 'Kreatinine (bloed)', 'KREAT enzym. (bloed)'
    'Bicarbinate': [],
    'BUN':         [],
    'pH':          [6848],                           # 'pH (bloed)'
    'Albumin':     [9975],                           # 'Albumine (imm.) (bloed)'
    'Bands':       [11586],                          # -- Staaf % (bloed)
    'Hemoglobin':  [9960],                           # 'Hb (bloed)'
    'BaseExcess':  []
}

In [ ]:
labs_df = []

for i, chunk in tqdm(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/numericitems.csv", usecols=['admissionid', 'itemid', 'value', 'measuredat', 'unitid'])):
    
    for lab, lab_ids in LABRESULTS.items():
        # Mask entries of one lab test
        is_lab = chunk.itemid.isin(lab_ids)
        
        if is_lab.any():
            
            # Rate conversions
            chunk.loc[chunk.unitid == 152, 'value'] *= chunk.value * 7.50061683  # conversion rate kPa to mmHg
            
            # Store lab measurements in new DataFrame
            lab_df = pd.DataFrame({
                'icustay_id': chunk.admissionid[is_lab].values,
                'charttime': time_within_admission(chunk.measuredat[is_lab], chunk.admissionid[is_lab]),
                'lab_id': lab,
                'valuenum': chunk.value[is_lab].values,
            })
            
            labs_df.append(lab_df)
        
    if i > MAX_CHUNKS:
        break
        
# Merge lab result DataFrames
labs_df = pd.concat(labs_df, axis=0).reset_index(drop=True)
labs_df.head()

---
## Urine Output

In [ ]:
URINE_OUTPUTS = [
    8794,  # UrineCAD
    8796,  # UrineSupraPubis
    8798,  # UrineSpontaan
    8800,  # UrineIncontinentie
    8803,  # UrineUP
    10743, # Nefrodrain li Uit
    10745, # Nefrodrain re Uit
    19921, # UrineSplint Li
    19922  # UrineSplint Re
]

In [ ]:
urine_outputs_df = []

for i, chunk in tqdm(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/numericitems.csv", usecols=['admissionid', 'itemid', 'value', 'measuredat'])):
    
    # which entries have urine output
    is_urine_output = chunk.itemid.isin(URINE_OUTPUTS)

    if is_urine_output.any():
        
        # Store lab measurements in new DataFrame
        urine_output_df = pd.DataFrame({
            'icustay_id': chunk.admissionid[is_urine_output].values,
            'charttime': time_within_admission(chunk.measuredat[is_urine_output], chunk.admissionid[is_urine_output]),
            'value': chunk.value[is_urine_output].values,
        })

        urine_outputs_df.append(urine_output_df)
        
    if i > MAX_CHUNKS:
        break
        
# Merge urine output DataFrames
urine_outputs_df = pd.concat(urine_outputs_df, axis=0).reset_index(drop=True)
urine_outputs_df.head()

---

## Vasopressors

Sources:
- Overview common vasopressor drugs for treatment of sepsis: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7333107/
- Vasopressors in UMCdb: [vasopressors_inotropes.ipynb](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/concepts/lifesupport/vasopressors_inotropes.ipynb), [vasopressors_inotropes.sql](https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/sql/common/vasopressors_inotropes.sql)
- Norepinephrine-equivalents: [get_vassopressor_mv.sql](https://github.com/LucaMD/SRL/blob/master/SEPSIS/MIMIC_sql/get_vassopressor_mv.sql)

Exclude *inotropes* which may be used to increase heart rate

In [ ]:
VASOPRESSORS = [
    7179,  # -- Dopamine (Inotropin)
    # 7178, -- Dobutamine (Dobutrex)           -> inotropic
    6818,  # -- Adrenaline (Epinefrine)        -> inotropic but also known to induce vasoconstriction
    7229,  # -- Noradrenaline (Norepinefrine)   
    # 7135, -- Isoprenaline (Isuprel)          -> inotropic
    # 7196, -- Enoximon (Perfan)               -> inotropic
    12467, # -- Terlipressine (Glypressin)
    13490, # -- Methyleenblauw IV (Methylthionide chloride)
    19929  # -- Fenylefrine
]

#### Extracting weight of patients
To compute the vasopressor dose in mcg/kg/min we need to know the weight of the patient in kg. We determine the approximate weight of the patient using the `weightgroup` entry in `admissions.csv` and the weight definitions in `WEIGHT_GROUPS` above.

In [ ]:
patient_weight = dict()

for i, chunk in read_csv(r"D:/AmsterdamUMCdb-v1.0.2/admissions.csv", usecols=['admissionid', 'weightgroup'], chunksize=10000000): # -> do all at once, this file is super small
    # Convert weightgroups, e.g. '70-79', to approximate weight (75)
    weights = chunk.weightgroup.transform(lambda adm: WEIGHT_GROUPS[adm] if adm in WEIGHT_GROUPS else 80)  # Assume 80 if unknown
    patient_weight = dict(zip(chunk.admissionid, weights))

In [ ]:
patient_weight[2] # in kgs

#### Computing vasopressor rate in mcg/kg/min

- ordercategoryid
    - 65 -> continuous IV perfusor
- doserateperkg:
    - Whether dose is already per kg -> 0/1
- doseunitid:
    - 10 -> mg
    - 11 -> µg
- doserateunitid:
    - 4 -> min
    - 5 -> uur

In [ ]:
vaso_cols = ['admissionid', 'itemid', 'start', 'stop', 'ordercategoryid', 'administeredunit', 'dose', 'doserateperkg', 'doseunitid', 'doserateunitid', 'rate']

vasopressors_df = []

for i, chunk in tqdm(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/drugitems.csv", usecols=vaso_cols)):
    # convert starttime and endtime to hours since start of admission
    chunk['start'] = time_within_admission(chunk.start, chunk.admissionid)
    chunk['stop'] = time_within_admission(chunk.stop, chunk.admissionid)
    
    # extract drugitems corresponding to vasopressors via continuous IV
    chunk = chunk[chunk.itemid.isin(VASOPRESSORS) & (chunk.rate > 0) & (chunk.ordercategoryid == 65)].copy()   
      
    # rate conversion
    per_kg = chunk.doserateperkg == 1
    is_mg = chunk.doseunitid == 10 
    per_hour = chunk.doserateunitid == 5
    weight_kg = chunk.admissionid.transform(lambda adm: patient_weight[adm] if adm in patient_weight else 80)
        
    chunk.loc[~per_kg, 'dose'] = chunk.dose / weight_kg
    chunk.loc[is_mg, 'dose'] = 1000 * chunk.dose
    chunk.loc[per_hour, 'dose'] = chunk.dose / 60
    
    # çonvert to norepinephrine equivalent dose
    # TODO: Terlipressine
    # TODO: Methyleenblauw IV
    chunk.loc[chunk.itemid == 7179, 'dose'] = 0.01 * chunk.dose   # Dopamine
    chunk.loc[chunk.itemid == 19929, 'dose'] = 0.45 * chunk.dose  # Fenylefrine
    
    # absolute start of admission is not known -> assume now
    admission_start = pd.Timestamp.now()

    # convert to DataFrame
    vasopressor_df = pd.DataFrame({
        'icustay_id': chunk.admissionid,
        'starttime': hours_to_absolute_time(admission_start, hours=chunk.start),
        'endtime': hours_to_absolute_time(admission_start, hours=chunk.stop),
        'mcgkgmin': chunk.dose
    })
    vasopressors_df.append(vasopressor_df)
    
    if i > MAX_CHUNKS:
        break
    
# merge
vasopressors_df = pd.concat(vasopressors_df, axis=0).reset_index(drop=True)
vasopressors_df.head()

In [ ]:
# save!
vasopressors_df.to_csv(OUT_DIR + 'vassopressors_mv_cohort.csv')

---

## Fluids

In [ ]:
IV_CATEGORIES = [
    # 25, # Injecties Haematologisch
    55, # Infuus - Crystalloid
    # 27, # Injecties Overig
    # 23, # Injecties CZS/Sedatie/Analgetica
    # 67, # Injecties Hormonen/Vitaminen/Mineralen
    65, # 2. Spuitpompen
    # 24, # Injecties Circulatie/Diuretica
    15, # Injecties Antimicrobiele middelen
    17, # Infuus - Colloid
    61, # Infuus - Bloedproducten
]

In [ ]:
fluid_cols = ['admissionid', 'itemid', 'item', 'start', 'stop', 'ordercategory', 'ordercategoryid', 'administered', 'administeredunitid']

iv_fluids_df = []

for i, chunk in tqdm(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/drugitems.csv", usecols=fluid_cols)):
    # convert starttime and endtime to hours since start of admission
    chunk['start'] = time_within_admission(chunk.start, chunk.admissionid)
    chunk['stop'] = time_within_admission(chunk.stop, chunk.admissionid)
        
    # extract drugitems corresponding to intravenous infusion in ml
    chunk = chunk[chunk.ordercategoryid.isin(IV_CATEGORIES) & (chunk.administeredunitid == 6)].copy()
        
    # TODO: tonicity
    
    # absolute start of admission is not known -> assume now
    admission_start = pd.Timestamp.now()

    # convert to DataFrame
    iv_fluid_df = pd.DataFrame({
        'icustay_id': chunk.admissionid,
        'starttime': hours_to_absolute_time(admission_start, hours=chunk.start),
        'endtime': hours_to_absolute_time(admission_start, hours=chunk.stop),
        'itemid': -1, # -1 as we'll be using the MIMIC preprocessing
        'ordercategoryname': chunk.ordercategory,
        'amountuom': 'ml',
        'amount': chunk.administered
    })
    iv_fluids_df.append(iv_fluid_df)
    
    if i > MAX_CHUNKS:
        break
    
# merge
iv_fluids_df = pd.concat(iv_fluids_df, axis=0).reset_index(drop=True)
iv_fluids_df.head()

In [ ]:
# save!
iv_fluids_df.to_csv(OUT_DIR + 'inputevents_mv_cohort.csv')

---

## Sepsis-3 cohort

### Suspicion of infection
#### Definitions

In [55]:
# see: https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/sql/diagnosis/reason_for_admission.sql
IS_SURGICAL = [
    13116, # D_Thoraxchirurgie_CABG en Klepchirurgie
    16671, # DMC_Thoraxchirurgie_CABG en Klepchirurgie
    13117, # D_Thoraxchirurgie_Cardio anders
    16672, # DMC_Thoraxchirurgie_Cardio anders
    13118, # D_Thoraxchirurgie_Aorta chirurgie
    16670, # DMC_Thoraxchirurgie_Aorta chirurgie
    13119, # D_Thoraxchirurgie_Pulmonale chirurgie
    16673, # DMC_Thoraxchirurgie_Pulmonale chirurgie
    13121, # D_Algemene chirurgie_Buikchirurgie
    16643, # DMC_Algemene chirurgie_Buikchirurgie
    13123, # D_Algemene chirurgie_Endocrinologische chirurgie
    16644, # DMC_Algemene chirurgie_Endocrinologische chirurgie
    13145, # D_Algemene chirurgie_KNO/Overige
    16645, # DMC_Algemene chirurgie_KNO/Overige
    13125, # D_Algemene chirurgie_Orthopedische chirurgie
    16646, # DMC_Algemene chirurgie_Orthopedische chirurgie
    13122, # D_Algemene chirurgie_Transplantatie chirurgie
    16647, # DMC_Algemene chirurgie_Transplantatie chirurgie
    13124, # D_Algemene chirurgie_Trauma
    16648, # DMC_Algemene chirurgie_Trauma
    13126, # D_Algemene chirurgie_Urogenitaal
    16649, # DMC_Algemene chirurgie_Urogenitaal
    13120, # D_Algemene chirurgie_Vaatchirurgie
    16650, # DMC_Algemene chirurgie_Vaatchirurgie
    13128, # D_Neurochirurgie _Vasculair chirurgisch
    16661, # DMC_Neurochirurgie _Vasculair chirurgisch
    13129, # D_Neurochirurgie _Tumor chirurgie
    16660, # DMC_Neurochirurgie _Tumor chirurgie
    13130, # D_Neurochirurgie_Overige
    16662, # DMC_Neurochirurgie_Overige
    18596, # Apache II Operatief  Gastr-intenstinaal
    18597, # Apache II Operatief Cardiovasculair
    18598, # Apache II Operatief Hematologisch
    18599, # Apache II Operatief Metabolisme
    18600, # Apache II Operatief Neurologisch
    18601, # Apache II Operatief Renaal
    18602, # Apache II Operatief Respiratoir
    17008, # APACHEIV Post-operative cardiovascular
    17009, # APACHEIV Post-operative gastro-intestinal
    17010, # APACHEIV Post-operative genitourinary
    17011, # APACHEIV Post-operative hematology
    17012, # APACHEIV Post-operative metabolic
    17013, # APACHEIV Post-operative musculoskeletal /skin
    17014, # APACHEIV Post-operative neurologic
    17015, # APACHEIV Post-operative respiratory
    17016, # APACHEIV Post-operative transplant
    17017  # APACHEIV Post-operative trauma
]

SEPSIS_ANTIBIOTICS = [
    6834, # Amikacine (Amukin)
    6847, # Amoxicilline (Clamoxyl/Flemoxin)
    6871, # Benzylpenicilline (Penicilline)
    6917, # Ceftazidim (Fortum)
    # 6919, # Cefotaxim (Claforan) -> prophylaxis
    6948, # Ciprofloxacine (Ciproxin)
    6953, # Rifampicine (Rifadin)
    6958, # Clindamycine (Dalacin)
    7044, # Tobramycine (Obracin)
    # 7064, # Vancomycine -> prophylaxis for valve surgery
    7123, # Imipenem (Tienam)
    7185, # Doxycycline (Vibramycine)
    # 7187, # Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
    # 7208, # Erythromycine (Erythrocine) -> often used for gastroparesis
    7227, # Flucloxacilline (Stafoxil/Floxapen)
    7231, # Fluconazol (Diflucan)
    7232, # Ganciclovir (Cymevene)
    7233, # Flucytosine (Ancotil)
    7235, # Gentamicine (Garamycin)
    7243, # Foscarnet trinatrium (Foscavir)
    7450, # Amfotericine B (Fungizone)
    # 7504, # X nader te bepalen # non-stock medication
    8127, # Meropenem (Meronem)
    8229, # Myambutol (ethambutol)
    8374, # Kinine dihydrocloride
    # 8375, # Immunoglobuline (Nanogam) -> not anbiotic
    # 8394, # Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
    8547, # Voriconazol(VFEND)
    # 9029, # Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
    9030, # Aztreonam (Azactam)
    9047, # Chlooramfenicol
    # 9075, # Fusidinezuur (Fucidin) -> prophylaxis
    9128, # Piperacilline (Pipcil)
    9133, # Ceftriaxon (Rocephin)
    # 9151, # Cefuroxim (Zinacef) -> often used for GI/transplant surgical prophylaxis
    # 9152, # Cefazoline (Kefzol) -> prophylaxis for cardiac surgery
    9458, # Caspofungine
    9542, # Itraconazol (Trisporal)
    # 9602, # Tetanusimmunoglobuline -> prophylaxis/not antibiotic
    12398, # Levofloxacine (Tavanic)
    12772, # Amfotericine B lipidencomplex  (Abelcet)
    15739, # Ecalta (Anidulafungine)
    16367, # Research Anidulafungin/placebo
    16368, # Research Caspofungin/placebo
    18675, # Amfotericine B in liposomen (Ambisome )
    19137, # Linezolid (Zyvoxid)
    19764, # Tigecycline (Tygacil)
    19773, # Daptomycine (Cubicin)
    20175  # Colistine
]

# Other antibiotics may be administered during sepsis treatment
# (but drug culture must be drawn!)
OTHER_ANTIBIOTICS = [
    7064, # Vancomycine -> prophylaxis for valve surgery
    7187, # Metronidazol (Flagyl) -> often used for GI surgical prophylaxis
    8394, # Co-Trimoxazol (Bactrimel) -> often prophylactic (unless high dose)
    9029, # Amoxicilline/Clavulaanzuur (Augmentin) -> often used for ENT surgical prophylaxis
    9151, # Cefuroxim (Zinacef) -> often used for GI surgical prophylaxis
    9152  # Cefazoline (Kefzol) -> prophylaxis
]

# Culture tests
CULTURE_TESTS = [
    9189, # Bloedkweken afnemen
    9190, # Cathetertipkweek afnemen
    9198, # Rectumkweek afnemen -> often used routinely
    9200, # Wondkweek afnemen
    9202, # Ascitesvochtkweek afnemen
    9205 # Legionella sneltest (urine)
]

# surgical admissions vs non-surgical admissions
re_sepsis_surg = r'sepsis|pneumoni|GI perforation|perforation/rupture|infection|abscess|GI Vascular ' \
                 r'ischemia|diverticular|appendectomy|peritonitis '

re_sepsis_med = r'sepsis|septic|infect|pneumoni|cholangitis|pancr|endocarditis|meningitis|GI ' \
                r'perforation|abces|abscess|darm ischaemie|GI vascular|fasciitis' + \
                r'|inflammatory|peritonitis'

#### 1. Were antibiotics administered during first 24h of admission?

In [56]:
antibiotics_df = []
for i, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/drugitems.csv", usecols=['admissionid', 'itemid', 'start'])):
    # were antibiotics administered within 24 hours?
    # remark: reason_for_admission.sql assumed admission to be the first 
    chunk['sepsis_antibiotics_bool'] = chunk.itemid.isin(SEPSIS_ANTIBIOTICS) & (time_within_admission(chunk.start, chunk.admissionid) < 24)  # first 24 hours
    chunk['other_antibiotics_bool'] = chunk.itemid.isin(OTHER_ANTIBIOTICS) & (time_within_admission(chunk.start, chunk.admissionid) < 24)    # Must also check for culture!
    
    # drop rows without antibiotics to save memory :)
    chunk = chunk.loc[chunk.sepsis_antibiotics_bool | chunk.other_antibiotics_bool, :]
    antibiotics_df.append(chunk)

antibiotics_df = pd.concat(antibiotics_df, axis=0)

100%|██████████████████████████████████████████████████████████████████████████▋| 22999/23106 [00:53<00:00, 433.56it/s]


#### 2. Was a positive blood culture found within first 6h?

In [57]:
culture_df = []
for i, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/procedureorderitems.csv", usecols=['admissionid', 'itemid', 'registeredat'])):
    # culture drawn?
    chunk['positive_blood_culture'] = chunk.itemid.isin(CULTURE_TESTS) & (time_within_admission(chunk.registeredat, chunk.admissionid) < 24)  # first 24 hours
    
    # drop rows with all False to save memory :)
    chunk = chunk.loc[chunk.positive_blood_culture, :]
    culture_df.append(chunk)

culture_df = pd.concat(culture_df, axis=0)

 99%|█████████████████████████████████████████████████████████████████████████▎| 22883/23106 [00:15<00:00, 1448.59it/s]


#### 3. Was there an infection/sepsis-related diagnosis?

In [58]:
# Based on code base from the AmsterdamUMCdb repository
# see: https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/sql/diagnosis/reason_for_admission.sql

def get_diagnosis(df):
    # drop prefixes from diagnosis name if APCAHE II/IV
    mask = df.itemid.isin([18669, 18671])
    df.loc[mask, 'value'] = df[mask].value.str.split(' - ').str[1]
    df['diagnosis'] = df.value

def is_surgical(df):
    df['surgical_adm'] = df.itemid.isin(IS_SURGICAL) | ((df.itemid == 18669) & ((df.valueid > 0) & (df.valueid <= 27))) | ((df.itemid == 18671) & ((df.valueid > 221) & (df.valueid <= 452)))

def sepsis_at_admission(df):
    df['known_no_sepsis'] = (df.itemid == 15808) & (df.valueid == 2)  # 15808 = 'opname sepsis'; 2 = explicitly no sepsis (to drop later)
    df['sepsis_on_admission'] = (df.itemid == 15808) & (df.valueid == 1)

In [59]:
diagnoses_df = []
for i, chunk in pbar(read_csv(r"D:/AmsterdamUMCdb-v1.0.2/listitems.csv", usecols=['admissionid', 'itemid', 'valueid', 'value', 'updatedat'])):   
    # was sepsis explicitly diagnosed?
    sepsis_at_admission(chunk) 
    
    # diagnosis related to infection/sepsis?
    get_diagnosis(chunk)
    is_surgical(chunk)
    surg_diag = chunk.surgical_adm & chunk.diagnosis.str.contains(re_sepsis_surg, na=False, flags=re.IGNORECASE)  # surgical admission with sepsis
    med_diag = ~chunk.surgical_adm & chunk.diagnosis.str.contains(re_sepsis_med, na=False, flags=re.IGNORECASE)   # medical admission with sepsis
    chunk['suspected_infection'] = surg_diag | med_diag
    
    # drop rows that aren't needed to save :)
    chunk = chunk.loc[chunk.suspected_infection | chunk.known_no_sepsis | chunk.sepsis_on_admission, :]
    diagnoses_df.append(chunk)

diagnoses_df = pd.concat(diagnoses_df, axis=0)

100%|████████████████████████████████████████████████████████████████████████████| 23106/23106 [05:48<00:00, 66.27it/s]


#### Get admissions with suspected infections

In [60]:
# admission ids with antibiotics
sepsis_antibiotics = set(antibiotics_df.admissionid[antibiotics_df.sepsis_antibiotics_bool].unique())
other_antibiotics = set(antibiotics_df.admissionid[antibiotics_df.other_antibiotics_bool].unique())

# admission ids with positive blood cultures
pos_blood_culture = set(culture_df.admissionid)

# admission ids with explicit diagnosis or suspicion of sepsis
sepsis_on_admission = set(diagnoses_df.admissionid[diagnoses_df.sepsis_on_admission].unique())
susp_infection = set(diagnoses_df.admissionid[diagnoses_df.suspected_infection].unique())
known_no_sepsis = set(diagnoses_df.admissionid[diagnoses_df.known_no_sepsis].unique())

# see: https://github.com/AmsterdamUMC/AmsterdamUMCdb/blob/master/amsterdamumcdb/cohorts.py
# suspicion of infection is quantified as either a known diagnosis of a major injection after/during a medical/surgical 
# admission OR when sepsis-related antibiotics are administered OR when a positive blood culture is found
adm_susp_infection = (susp_infection | sepsis_on_admission | sepsis_antibiotics | (other_antibiotics & pos_blood_culture)) - known_no_sepsis

In [61]:
print('With suspected infection:', len(adm_susp_infection))

With suspected infection: 3769


#### 4. Estimate infection time

We define the suspected infection time as the earliest moment of administering antibiotics, diagnosis of sepsis, or where a positive blood culture is found:

In [62]:
susp_infection_time_poe = {}
for admissionid in tqdm(adm_susp_infection):
    # determine earliest time where infection was suspected
    # i.e. earliest time of antibiotics OR culture found OR sepsis diagnosis
    times = antibiotics_df.start[antibiotics_df.admissionid == admissionid].tolist()  # empty if no antibiotics received
    times += culture_df.registeredat[culture_df.admissionid == admissionid].tolist()  # empty if no positive culture found
    times += diagnoses_df.updatedat[diagnoses_df.admissionid == admissionid].tolist() # empty if no explicit documented diagnosis
    min_time = min(times)
    
    # Compute suspected infection time in hours relative to start of admission
    starttime = start_of_admission[admissionid]
    susp_infection_time_poe[admissionid] = (min_time - starttime) / (1000 * 60 * 60)
    
susp_infection_time_poe[8216]

100%|████████████████████████████████████████████████████████████████████████████| 3769/3769 [00:02<00:00, 1470.90it/s]


14.366666666666667

In [ ]:
## Limit to patients with sofa > 2